In [1]:
!pip install opencv-python==4.1.0.25

     |████████████████████████████████| 26.6MB 105kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

IMAGE_PATH = "./cat.jpg"

model = tf.keras.applications.VGG16(weights="imagenet", include_top=True)

input_shape = (224, 224)

img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=input_shape)
img = tf.keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)

data = (preprocess_input(img), None)

553467904/553467096 [==============================] - 10s 0us/step


In [4]:
def resize_activations(enhanced_model_output, input_shape):
    """Utility function to resize a given tensor

    Args:
        enhanced_model_output (tf.Tensor): 4D-Tensor with shape (batch_size, H, W, K)
        input_shape (Tuple[int, int]): shape of the input, e.g. (224, 224)

    Returns
        tensor (tf.Tensor): 4D-Tensor with shape (batch_size, K, H, W)
    """

    resized_activations = list()

    for j in range(enhanced_model_output.shape[0]):
        acts = list()
        for i in range(enhanced_model_output.shape[-1]):
            acts.append(
                transform.resize(enhanced_model_output[j, ..., i], input_shape, preserve_range=True)
            )
        resized_activations.append(np.array(acts))

    return tf.convert_to_tensor(np.array(resized_activations), dtype=tf.float32)

def normalize_activations(tensor):
    """Utility function to normalize a given tensor

    Args:
        tensor (tf.Tensor): 4D-Tensor with shape (batch_size, K, H, W)

    Returns:
        tf.Tensor: 4D-Tensor with shape (batch_size, K, H, W)
    """

    tensors = list()

    # goes through each image
    for i in range(tensor.shape[0]):
        flattened = tf.reshape(tensor[i], (tensor[i].shape[0], -1))

        max_a = tf.math.reduce_max(flattened, axis=1)
        min_a = tf.math.reduce_min(flattened, axis=1)

        diffs = tf.where(max_a > min_a, max_a - min_a, 1)

        normalized_tensor = (tensor[i] - tf.reshape(min_a, (-1, 1, 1))) / tf.reshape(diffs, (-1, 1, 1))

        tensors.append(normalized_tensor)

    return tf.stack(tensors, axis=0)

In [5]:
def get_last_convolutional_layer_name(model):
    """
    Search for the last convolutional layer to perform Score-CAM, as stated
    in section 4.1 in the original paper.

    Args:
        model (tf.keras.Model): tf.keras model to inspect

    Returns:
        str: Name of the target layer
    """
    for layer in reversed(model.layers):
        # Select closest 4D layer to the end of the network.
        if len(layer.output_shape) == 4 and layer.name.count('conv') > 0:
            return layer.name

    raise ValueError(
        "Model does not seem to contain 4D layer. Grad CAM cannot be applied."
    )

In [6]:
from skimage import transform

In [7]:
images, _ = data
batch_size = images.shape[0]

# according to section 4.1 of paper, we need the last convolutional layer
layer_name = get_last_convolutional_layer_name(model)

# normalize feature maps, calculate masks and compute the
# output score
# weights, maps = get_filters(
#     model, images, layer_name, 281, input_shape
# )

conv_model = tf.keras.Model(
    inputs=model.input,
    outputs=model.get_layer(layer_name).output
)

softmax_model = tf.keras.models.Model(
    [model.inputs], [model.outputs]
)

inputs = tf.cast(images, tf.float32)

conv_output = conv_model.predict(inputs)
resized_conv_output = resize_activations(conv_output, input_shape)

In [8]:
normalized_maps = normalize_activations(resized_conv_output) # shape (batch_size, K, H, W)
shape = normalized_maps.shape

In [9]:
normalized_maps = tf.reshape(normalized_maps, (shape[1], shape[2], shape[3], shape[0]))

In [10]:
# (512, 224, 224, 1) * (512, 224, 224, 3)
masked_images = tf.math.multiply(normalized_maps, tf.tile(inputs, (normalized_maps.shape[0], 1, 1, 1)))

In [11]:
classes_activation_scale = softmax_model.predict(masked_images)

In [12]:
# return the output only for the given class
weights = classes_activation_scale[0][:, 281] # shape (K,)

In [13]:
weights = weights.reshape((-1, 1, 1, batch_size)) # shape (K, 1, 1, 1)

In [14]:
weights.shape

(512, 1, 1, 1)

In [15]:
normalized_maps.shape

TensorShape([512, 224, 224, 1])

In [16]:
cam = tf.math.multiply(weights, normalized_maps)

In [17]:
# relu
cam = tf.math.reduce_max(cam, axis=0)
relu_cam = tf.where(cam > 0, cam, 0)

In [18]:
relu_cam.shape

TensorShape([224, 224, 1])

In [19]:
relu_cam = tf.reshape(relu_cam, (relu_cam.shape[2], relu_cam.shape[0], relu_cam.shape[1]))

In [20]:
relu_cam.shape

TensorShape([1, 224, 224])

In [21]:
from pathlib import Path
import cv2

def save_rgb(image, output_dir, output_name):
    """
    Save a 3D Numpy array (H, W, 3) as an image.

    Args:
        image (numpy.ndarray): Image to save
        output_dir (str): Output directory
        output_name (str): Output name
    """
    Path.mkdir(Path(output_dir), parents=True, exist_ok=True)

    cv2.imwrite(
        str(Path(output_dir) / output_name), cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    )

In [22]:
def image_to_uint_255(image):
    """
    Convert float images to int 0-255 images.
    Args:
        image (numpy.ndarray): Input image. Can be either [0, 255], [0, 1], [-1, 1]
    Returns:
        numpy.ndarray:
    """
    if image.dtype == np.uint8:
        return image

    if image.max() > 1:
        return image.astype("uint8")

    if image.min() < 0:
        image = (image + 1.0) / 2.0

    return (image * 255).astype("uint8")

def heatmap_display(
    heatmap, original_image, colormap=cv2.COLORMAP_JET, image_weight=0.7
):
    """
    Apply a heatmap (as an np.ndarray) on top of an original image.

    Args:
        heatmap (numpy.ndarray): Array corresponding to the heatmap
        original_image (numpy.ndarray): Image on which we apply the heatmap
        colormap (int): OpenCV Colormap to use for heatmap visualization
        image_weight (float): An optional `float` value in range [0,1] indicating the weight of
            the input image to be overlaying the calculated attribution maps. Defaults to `0.7`

    Returns:
        np.ndarray: Original image with heatmap applied
    """
    heatmap = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))

    image = image_to_uint_255(original_image)

    heatmap = (heatmap - np.min(heatmap)) / (heatmap.max() - heatmap.min())

    heatmap = cv2.applyColorMap(
        cv2.cvtColor((heatmap * 255).astype("uint8"), cv2.COLOR_GRAY2BGR), colormap
    )

    output = cv2.addWeighted(
        cv2.cvtColor(image, cv2.COLOR_RGB2BGR), image_weight, heatmap, 1, 0
    )

    return cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

In [23]:
colormap = cv2.COLORMAP_JET
image_weight = 0.7

heatmaps = np.array(
    [
    # not showing the actual image if image_weight=0
    heatmap_display(_cam.numpy(), image, colormap, image_weight)
    for _cam, image in zip(relu_cam, img)
    ]
)

In [24]:
save_rgb(heatmaps[0], ".", "score_cam.png")